## Week 2:
 
New Topics:

 - Resources for help with coding 
 - Creating a reproducible workflow
 - Merging _pandas_ DataFrames
 - Additional _pandas_ methods 
 

Coding Tasks:

Start a new Jupyter Notebook to complete these tasks. This week, you'll be combining two different datasets.

First, you'll work with a dataset containing the number of primary care physicians per county for each county in the United States. It was obtained from the Area Health Resources File, published by the [Health Resources and Services Administration](https://data.hrsa.gov/topics/health-workforce/ahrf). This data is contained in the file `primary_care_physicians.csv`.

Second, the file `population_by_county.csv` contains the Census Bureau's 2019 population estimates for each US County. It also contains a column `urban`. The `urban` column uses data from the National Bureau of Economic Research to classify each county as either urban or rural. The U.S. Office of Management and Budget designates counties as metropolitan (a core urban area of 50,000 or more population), micropolitan (an urban core of at least 10,000 but less than 50,000 population), or neither. Here, a county is considered "urban" if it is part of a metropolitan or micropolitan area and "rural" if it is not.

 1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`. 

In [3]:
import pandas as pd
physicians = pd.read_csv('../data/primary_care_physicians.csv')
physicians

,FIPS,state,county,primary_care_physicians
0,1001,Alabama,Autauga,26.0
1,1003,Alabama,Baldwin,153.0
2,1005,Alabama,Barbour,8.0
3,1007,Alabama,Bibb,12.0
4,1009,Alabama,Blount,12.0
...,...,...,...,...
3225,72151,Puerto Rico,Yabucoa,5.0
3226,72153,Puerto Rico,Yauco,43.0
3227,78010,US Virgin Islands,St. Croix,14.0
3228,78020,US Virgin Islands,St. John,1.0


2. Filter physicians down to just the counties in Tennessee. Save the filtered dataframe back to physicians. Verify that the resulting dataframe has 95 rows.

In [4]:
physicians = physicians.loc[physicians['state'] == 'Tennessee']
physicians.shape

(95, 4)

3. Look at the distribution of the number of primary care physicians. What do you notice?

In [5]:
physicians

,FIPS,state,county,primary_care_physicians
2432,47001,Tennessee,Anderson,39.0
2433,47003,Tennessee,Bedford,15.0
2434,47005,Tennessee,Benton,3.0
2435,47007,Tennessee,Bledsoe,1.0
2436,47009,Tennessee,Blount,90.0
...,...,...,...,...
2522,47181,Tennessee,Wayne,5.0
2523,47183,Tennessee,Weakley,18.0
2524,47185,Tennessee,White,9.0
2525,47187,Tennessee,Williamson,338.0


4. Now, import the population by county dataset (population_by_county.csv) into a DataFrame named population.

In [6]:
population = pd.read_csv('../data/population_by_county.csv')
population

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural
...,...,...,...,...,...
3197,47033,14399,Crockett County,TENNESSEE,Rural
3198,47095,7401,Lake County,TENNESSEE,Rural
3199,47093,461104,Knox County,TENNESSEE,Urban
3200,53005,197518,Benton County,WASHINGTON,Urban


5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.

In [7]:
population['county'] = population['county'].str.split(' ', expand = True)[0]
population.head()

,FIPS,population,county,state,urban
0,17051,21565,Fayette,ILLINOIS,Rural
1,17107,29003,Logan,ILLINOIS,Rural
2,17165,23994,Saline,ILLINOIS,Rural
3,17097,701473,Lake,ILLINOIS,Urban
4,17127,14219,Massac,ILLINOIS,Rural


In [8]:
population.loc[population['state'] == 'TENNESSEE']
physicians = pd.merge(left = physicians,
         right = population[['county','population', 'urban']])
physicians

,FIPS,state,county,primary_care_physicians,population,urban
0,47001,Tennessee,Anderson,39.0,7835,Rural
1,47001,Tennessee,Anderson,39.0,22409,Rural
2,47001,Tennessee,Anderson,39.0,76061,Urban
3,47001,Tennessee,Anderson,39.0,198064,Urban
4,47001,Tennessee,Anderson,39.0,57810,Rural
...,...,...,...,...,...,...
631,47187,Tennessee,Williamson,338.0,225389,Urban
632,47189,Tennessee,Wilson,43.0,49173,Urban
633,47189,Tennessee,Wilson,43.0,8688,Rural
634,47189,Tennessee,Wilson,43.0,136666,Urban


6. How many Tennessee counties are considered urban?

In [9]:
physicians['urban'].value_counts()

Rural    404
Urban    232
Name: urban, dtype: int64

7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group?

In [12]:
res_per_physician = physicians['population'] / physicians['primary_care_physicians']
res_per_physician 

0       200.897436
1       574.589744
2      1950.282051
3      5078.564103
4      1482.307692
          ...     
631     666.831361
632    1143.558140
633     202.046512
634    3178.279070
635    1893.837209
Length: 636, dtype: float64

In [16]:
adequate = physicians.loc[(physicians['population'] / physicians['primary_care_physicians']) < 1500]
moderately_inadequate = physicians.loc[(physicians['population'] / physicians['primary_care_physicians'] >= 1500) | 
                                       (physicians['population'] / physicians['primary_care_physicians'] <= 3500)]
low_inadequate = physicians.loc[(physicians['population'] / physicians['primary_care_physicians']) > 3500]

In [25]:
num = len(adequate['county'])
moderately_inadequate['county']
low_inadequate['county']

239

8. Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?

In [30]:
adequate['urban'].value_counts()

Rural    177
Urban     62
Name: urban, dtype: int64

In [31]:
moderately_inadequate['urban'].value_counts()

Rural    404
Urban    232
Name: urban, dtype: int64

In [32]:
low_inadequate['urban'].value_counts()

Rural    135
Urban    117
Name: urban, dtype: int64